In [2]:
import chromadb
import ollama

In [3]:
chroma_client = chromadb.Client()


In [ ]:
#chroma_client.delete_collection(name="feedback_collection") # Delete a collection and all associated embeddings, documents, and metadata. ⚠️ This is destructive and not reversible
#chroma_client.delete_collection(name="python_collection") # Delete a collection and all associated embeddings, documents, and metadata. ⚠️ This is destructive and not reversible


In [4]:
feedback_collection = chroma_client.create_collection(name="feedback_collection")

In [6]:
import re
import os

base_path = "/Users/valeriaenriquezlimon/Documents/meta-h-generator/llm-metaheuristics/experiments-custumhys"
folder_name = "ollama_output_Rastrigin(5)_20241121_000015"
file_name = "execution_optuna_result_0.txt"
metaheuristic_file  = "/Users/valeriaenriquezlimon/Documents/meta-h-generator/llm-metaheuristics/experiments-custumhys/ollama_output_Rastrigin(5)_20241121_000015/execution_iteration_0.py"

# Construct the file path
file_path = os.path.join(base_path, folder_name, file_name)
with open(file_path, 'r', encoding='utf-8') as file:
    code_file = file.read()


with open(metaheuristic_file, 'r', encoding='utf-8') as file:
    metaheuristic_code_file = file.read()

hyperparameters_pattern = r"Mejores hiperparámetros encontrados:\n({.*?})"
performance_pattern = r"Mejor rendimiento encontrado:\n([\d.]+)"

hyperparameters_match = re.search(hyperparameters_pattern, code_file, re.DOTALL)
hyperparameters_dict = eval(hyperparameters_match.group(1)) if hyperparameters_match else None

performance_match = re.search(performance_pattern, code_file)
performance_found = float(performance_match.group(1)) if performance_match else None

if hyperparameters_dict:
    radius = float(hyperparameters_dict.get('radius', 0))
    angle = float(hyperparameters_dict.get('angle', 0))
    sigma = float(hyperparameters_dict.get('sigma', 0))
    factor = float(hyperparameters_dict.get('factor', 0))
    self_conf = float(hyperparameters_dict.get('self_conf', 0))
    swarm_conf = float(hyperparameters_dict.get('swarm_conf', 0))
    version = hyperparameters_dict.get('version', '')
    distribution = hyperparameters_dict.get('distribution', '')

    
    print(f"radius: {radius}")
    print(f"angle: {angle}")
    print(f"sigma: {sigma}")
    print(f"factor: {factor}")
    print(f"self_conf: {self_conf}")
    print(f"swarm_conf: {swarm_conf}")
    print(f"version: {version}")
    print(f"distribution: {distribution}")

print(f"Performance found: {performance_found}")


radius: 0.5405117232957318
angle: 14.408693167938573
sigma: 0.046498549162240936
factor: 0.11779253652571474
self_conf: 2.951026835950703
swarm_conf: 2.7883600653077947
version: inertial
distribution: uniform
Performance found: 7.955976169162414


In [7]:
number_iteration = "id_0"
parameters  = f"""radius": {radius},"angle": {angle},"sigma": {sigma},"factor": {factor},"self_conf": {self_conf},"swarm_conf": {swarm_conf},"version": {version},"distribution": {distribution}, "performance_found": {performance_found}"""
print(parameters)
feedback_collection.add(
    documents=[metaheuristic_code_file, parameters],
    metadatas=[{'source': 'metaheuristic_code_file'}, {'source': 'parameters'}],
    ids=["id_0", "id_1"]
)

radius": 0.5405117232957318,"angle": 14.408693167938573,"sigma": 0.046498549162240936,"factor": 0.11779253652571474,"self_conf": 2.951026835950703,"swarm_conf": 2.7883600653077947,"version": inertial,"distribution": uniform, "performance_found": 7.955976169162414


In [8]:
python_collection = chroma_client.create_collection(name="python_collection")


In [9]:
import os
import ollama  # Replace with the actual library you use for embeddings

# Define the directory containing Python files
python_files_directory = '/Users/valeriaenriquezlimon/Documents/meta-h-generator/llm-metaheuristics/metaheuristic_builder'

# Iterate through all items in the directory
for d in os.listdir(python_files_directory):
    file_path = os.path.join(python_files_directory, d)  # Full path to the file
    if os.path.isfile(file_path):  # Check if it's a file
        try:
            # Open and read the file
            with open(file_path, 'r') as file:
                file_content = file.read()

            # Generate embeddings for the file content
            response = ollama.embeddings(model="all-minilm:latest", prompt=file_content)
            embedding = response.get("embedding")

            # Add embedding to the collection if valid
            if embedding:
                python_collection.add(
                    ids=[d],
                    embeddings=[embedding],
                    documents=[file_content],
                    metadatas=[{"filename": d}]
                )
                print(f"Added {d} to the collection")
            else:
                print(f"Warning: Empty embedding generated for {d}")

        except Exception as e:
            print(f"Error processing {d}: {e}")
    else:
        # Skip if it's not a file
        print(f"Skipping {d}, not a file")


Added parameters_to_take.txt to the collection
Added metaheuristic_template.txt to the collection
Added optuna_template.txt to the collection
Added operators.txt to the collection


In [7]:
python_collection.peek()

{'ids': ['template.txt',
  'parameters_to_take.txt',
  'optuna_template.txt',
  'operators.txt'],
 'embeddings': array([[ 0.05051658,  0.07573234, -0.11015741, ..., -0.00664864,
          0.06657533, -0.12182963],
        [-0.01380775, -0.07297242, -0.04008463, ...,  0.02461663,
         -0.08961589, -0.1428356 ],
        [ 0.07156529,  0.10162532, -0.2259855 , ..., -0.04780288,
         -0.03705836,  0.01757822],
        [-0.0007704 ,  0.01256534, -0.07165639, ...,  0.00121322,
         -0.02565598,  0.03352246]]),
 'documents': ["When creating a metaheuristic take into account: You should NOT use any markdown code or use the triple backticks  (```) anywhere in your response, \nall outputs must be plain text. Use only the benchmark_function and its' dimension provided. \nRemeber that you should add more agents depending on the size of the dimension, the default agents are 2, however if there is a dimension as 5, you should add 2 more agents, and so on and so forth. \nIf there is a 10 

In [47]:
feedback_collection.peek()

{'ids': ['id_0', 'id_1'],
 'embeddings': array([[ 1.89556424e-02,  5.34033887e-02, -6.45390451e-02,
         -2.64242757e-04, -1.48799866e-02, -9.58701447e-02,
         -2.12539975e-02,  1.05552509e-01, -1.06204331e-01,
          4.38341796e-02,  6.61895610e-05, -9.30237100e-02,
          4.61782105e-02,  1.54377858e-03, -7.56902620e-02,
          2.40415544e-03, -1.47583932e-02, -5.56412041e-02,
          1.14936277e-03, -8.49774480e-02, -1.15783727e-02,
          3.01029757e-02,  7.42062554e-02,  3.82375001e-04,
         -3.41604836e-02, -1.07110348e-02, -3.55581492e-02,
          4.18012962e-02,  2.40666457e-02, -3.68813090e-02,
          8.44583437e-02,  5.77159040e-02, -5.47397807e-02,
         -1.54123958e-02,  6.01436533e-02,  2.56502144e-02,
         -1.27162009e-01, -7.87758734e-03,  1.45498812e-02,
          4.53483909e-02,  4.97481450e-02,  8.21180344e-02,
          4.16935720e-02, -1.32607445e-01,  4.00925912e-02,
          2.09104884e-02, -3.59574258e-02,  4.19869721e-02,


In [14]:
# all-minilm:latest
output = ollama.embeddings(
        prompt="give me parameters found, becuase those are the ones with the best performance",
        model="all-minilm:latest",
)
results = feedback_collection.query (
        query_embeddings=[output["embedding"]],
        n_results=1
)
data = results['documents'][0][0]
print(data)



radius": 0.5405117232957318,"angle": 14.408693167938573,"sigma": 0.046498549162240936,"factor": 0.11779253652571474,"self_conf": 2.951026835950703,"swarm_conf": 2.7883600653077947,"version": inertial,"distribution": uniform, "performance_found": 7.955976169162414


In [17]:

# Probably not useful
# all-minilm:latest
output_metaheuristics = ollama.embeddings(
        prompt="give me the metaheuristic",
        model="all-minilm:latest",
)
results_metaheuristics = python_collection.query (
        query_embeddings=[output_metaheuristics["embedding"]],
        n_results=2
)
#data_metaheuristics = results_metaheuristics['documents'][0][0]
results_metaheuristics
#print(data_metaheuristics)


{'ids': [['metaheuristic_template.txt', 'optuna_template.txt']],
 'embeddings': None,
 'documents': [['# This is the metaheuristic template:\n\nFormat your response exaclty as follows.  \nDo not write anything before this format: \n      \n# Name: [Your chosen name for the metaheuristic]\n# Code:\nimport sys\nfrom pathlib import Path\n\nproject_dir = Path(__file__).resolve().parents[2] # Remember to write well this line: \'project_dir = Path(__file__).resolve().parents[2]\'\nsys.path.insert(0, str(project_dir))\nimport benchmark_func as bf\nimport metaheuristic as mh\n\nfun = bf.{self.benchmark_function}({self.dimensions}) # This is the selected problem, the problem may vary depending on the case.\nprob = fun.get_formatted_problem()\n\nheur = [\n    (  # Search operator 1\n        \'[operator_name]\',\n        {\n            \'parameter1\': value1,\n            \'parameter2\': value2,\n            more parameters as needed\n        },\n        \'[selector_name]\'\n    ),\n    (\n      

In [ ]:
ollama.list

<bound method Client.list of <ollama._client.Client object at 0x14720ed80>>

In [ ]:
output = ollama.generate(
   model = "myqwen2.5:latest",
   prompt = f"""Using these parametes {data} and this metaheuristic data {data_metaheuristics}
   , create another metaheuristic with a better performance, you must use the given parameters in {data} because
   those are the ones that were genereted in an optimized function,
   do not use markdown code anywhere in the project, take into account the problem fun = bf.Rastrigin(5).
   """
)

print(output['response'])

# Name: EnhancedInertialSwarmOptimization

# Code:
import sys
from pathlib import Path

project_dir = Path(__file__).resolve().parents[2] # Remember to write well this line: 'project_dir = Path(__file__).resolve().parents[2]'
sys.path.insert(0, str(project_dir))
import benchmark_func as bf
import metaheuristic as mh

fun = bf.Rastrigin(5) # This is the selected problem, the problem may vary depending on the case.
prob = fun.get_formatted_problem()

heur = [
    (  # Search operator 1
        'swarm_dynamic',
        {
            'factor': 0.11779253652571474,
            'self_conf': 2.951026835950703,
            'swarm_conf': 2.7883600653077947,
            'version': "inertial",
            'distribution': "uniform"
        },
        'probabilistic'
    ),
    (
        'local_random_walk',
        {
            'probability': 0.75,
            'scale': 1.0,
            'distribution': "uniform"
        },
        'probabilistic'
    )
]

met = mh.Metaheuristic(prob, heur, num_ite